In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from task2_funcs import *
from tabulate import tabulate

# Task 2: Consultancy for opening a restaurant in the city of Philadelphia

<!-- Task 2 is more open in nature as there is no specific target. We don’t expect you to analyse all aspects of the problem. You can decide yourself on which approaches, summary statistics or analysis procedures you focus on. Grading will be based on scientific correctness, originality and presentation. -->

## Introduction

The goal of this report is to analyse customer review data to better understand the regional market so that the business is more likely to achieve success for the opening of their restaurant in Philadelphia. In particular, we aim to answer the following questions:

1. An insight on what restaurant consumers generally seem to like (for example in terms of food, service, location, etc…).
2. An analysis of the evolution of food trends in the area over time, in terms of consumer preferences. Do the preferences evolve over time, or do they seem stable?
3. Imagine you have to present your findings to the business owner and his investors. What advice would you give to the new business, based on your findings?

Given the nature of the task, we'll focus on customer reviews which concern restaurants located in the city of Philadelphia. First, let's take a look at the data to see how we can approach this task.

## Extracting restaurant reviews in Philadelphia
Before starting the analysis, we'll have to obtain the relevant review data. Let's start by importing the data and then inspecting their structures. It is worth mentioning that the reviews in the test data (`ATML2024_reviews_test.csv`) are NOT used in this analysis, since they don't contain the ratings by customers for the businesses which will hinder our ability to know the preferences of restaurants in Philadelphia.

In [2]:
# Importing the dataset
reviews_df = pd.read_csv("datasets/ATML2024_reviews_train.csv")
users_df = pd.read_csv("datasets/ATML2024_users.csv")
business_df = pd.read_csv("datasets/ATML2024_businesses.csv")

### Glimpsing at the data
Below cells show the data types of the columns as well as the first 5 rows of each dataset. Based on the output, in order to extract reviews about restaurants in Philadelphia, we can first filter out businesses who are based in Philadelphia under the `city` column, and then look at those whose categories include restaurants. One crucial thing to note that, however, is that due to the textual nature of the data, there's no guarantee that the `city` column is free of typos or has standardised how Philadelphia is referred to. For example, the city is sometimes referred to as Philly. We shall inspect this column more in detail to ensure that we include all the restaurant reviews in Philadelphia (or at least we don't miss out too much because of typos).

We can also notice that some columns aren't in the correct data types and will need to be changed if they're to be used in the following analysis. For instance, the date-related columns (`date` in `reviews_df` and `user_since` in `users_df`) are wrongly marked as `object`, and the `premium_account` column is just a string of years concatenated together which might pose some troubles if we'd like to look like the number of premium users by year. But for now let's focus on filtering restaurant reviews in Philadelphia.

In [3]:
print(reviews_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050000 entries, 0 to 1049999
Data columns (total 9 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   id           1050000 non-null  int64 
 1   user_id      1050000 non-null  object
 2   business_id  1050000 non-null  object
 3   rating       1050000 non-null  int64 
 4   useful       1050000 non-null  int64 
 5   funny        1050000 non-null  int64 
 6   cool         1050000 non-null  int64 
 7   text         1050000 non-null  object
 8   date         1050000 non-null  object
dtypes: int64(5), object(4)
memory usage: 72.1+ MB
None


In [4]:
print(tabulate(reviews_df.head(), headers = "keys", tablefmt='orgtbl', showindex=False))

|   id | user_id                | business_id            |   rating |   useful |   funny |   cool | text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | date    

In [5]:
print(users_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 747468 entries, 0 to 747467
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   user_id             747468 non-null  object 
 1   name                747457 non-null  object 
 2   user_since          747468 non-null  object 
 3   useful              747468 non-null  float64
 4   funny               747468 non-null  float64
 5   cool                747468 non-null  float64
 6   premium_account     57420 non-null   object 
 7   friends             747468 non-null  float64
 8   fans                747468 non-null  float64
 9   compliment_hot      747468 non-null  float64
 10  compliment_more     747468 non-null  float64
 11  compliment_profile  747468 non-null  float64
 12  compliment_cute     747468 non-null  float64
 13  compliment_list     747468 non-null  float64
 14  compliment_note     747468 non-null  float64
 15  compliment_plain    747468 non-nul

In [6]:
print(tabulate(users_df.head(), headers='keys', tablefmt='orgtbl', showindex=False))

| user_id                | name   | user_since          |   useful |   funny |   cool | premium_account                                                   |   friends |   fans |   compliment_hot |   compliment_more |   compliment_profile |   compliment_cute |   compliment_list |   compliment_note |   compliment_plain |   compliment_cool |   compliment_funny |   compliment_writer |
|------------------------+--------+---------------------+----------+---------+--------+-------------------------------------------------------------------+-----------+--------+------------------+-------------------+----------------------+-------------------+-------------------+-------------------+--------------------+-------------------+--------------------+---------------------|
| w7IdXgBVXKjZS5UYDO8cVq | Walker | 2007-01-25 16:47:26 |     7217 |    1259 |   5994 | 2007                                                              |     14995 |    267 |              250 |                65 |                   

In [7]:
print(business_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138210 entries, 0 to 138209
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   business_id  138210 non-null  object 
 1   name         138210 non-null  object 
 2   address      133772 non-null  object 
 3   city         138210 non-null  object 
 4   state        138210 non-null  object 
 5   postal_code  138145 non-null  object 
 6   latitude     138210 non-null  float64
 7   longitude    138210 non-null  float64
 8   attributes   126589 non-null  object 
 9   categories   138136 non-null  object 
 10  hours        117852 non-null  object 
dtypes: float64(2), object(9)
memory usage: 11.6+ MB
None


In [8]:
print(tabulate(business_df.head(), headers = "keys", tablefmt='orgtbl', showindex = False))

| business_id            | name                     | address                         | city         | state   |   postal_code |   latitude |   longitude | attributes                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | categories                                                                         | hours                                                                                                                                     

### Is it Philadelphia, or ...?
We now take a closer look to how Philadelphia might be referred to in the dataset. From the regex search result below, even just by using the full name of the city, there already exist 6 ways to which the city is referred. Moreover, the below list doesn't include typos nor or nicknames of the city. Therefore, we'll need to normalise the different ways Philadelphia is called in the `business_df` dataset, and we need a way to deal with a non-exhaustive list of alias of Philadelphia. 

In [9]:
import re

# Checking how Philadelphia might be referred to
business_pa = business_df.query("state == 'PA'")  # Since Philadelphia is located in the state of Pennsylvania
business_pa.loc[:, 'city'] = business_pa['city'].str.lower()  # Avoid case-sensitivity issues in string matching later
unique_cities = business_pa['city'].unique()  
philly_matches = [re.search(r"philadelphia", city) is not None for city in unique_cities]
print(unique_cities[philly_matches])

['philadelphia' 'southwest philadelphia' 'philadelphia pa'
 'west philadelphia' 'philadelphia (northeast philly)' 'philadelphia ']


To compare similarities between some strings and "Philadelphia", we can use the Jaro similarity which ranges from 0 (totally dissimilar) to 1 (exact match) between two strings. Mathematically, Jaro similarity $sim_j$ between two strings $s_1 \; \text{and} \; s_2$ is defined as below (a more detailed discussion of the Jaro similairty and its variants can be found [here](https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance)):

\begin{align}
sim_j = 
\begin{cases}
0 \quad &\text{if} \; m = 0 \\
\frac{1}{3}(\frac{m}{|s_1|} + \frac{m}{|s_2|} + \frac{m-t}{m}) \; &\text{otherwise}
\end{cases},
\end{align}

where $|s_i|$ is the length of the string $s_i$, $m$ is the number of matching characters (characters in $s_1 \; \text{and} \; s_2$ are matching only if they're at most $[\frac{\text{max}(|s_1|, |s_2|)}{2}] - 1$ characters apart), and $t$ is the number of transpositions (i.e. swapping the positions of two characters) which is calculated as number of matching characters not being in the correct order divided by two.

Admittedly, there exist other string distance measures such as the Levenshtein distance as briefly mentioned in class. Nevertheless, we will use Jaro similarity because its output always ranges from 0 to 1 no matter the lengths of the two strings under comparison which makes thresholding easier. Moreover, Jaro similarity satisfies the mathematical definition of a distance metric, whereas its variant Jaro-Winkler similarity violates the triangle inequality.

We now move on to testing how many cities will be replaced by "philadelphia" depending on the threshold of Jaro similarity score. The python package `jellyfish` has already implemented a function for calculating Jaro similarity between two strings, and we shall define a function for replacing city names in the dataset based on this. Note that we've converted the unique city names in the dataset to be in lowercases since the implementation in `jellyfish` is case-sensitive, i.e. same character in different cases will not be counted as matching characters.

Starting with an arbitrary threshold of 0.6, we notice that matches with scores lower than 0.7 are wrongly replaced with 'philadelphia' while in reality they are genuinely referring to another locale in the state of Pennsylvania. We therefore should not include these matches in our final dataset. On the other hand, the algorithm manages to capture the 11 different alias of Philadelphia and replace them accordingly once the Jaro score is at least 0.7. Indeed, all the matching results seem to be about the city of Philadelphia, whether it be typos or nicknames of the city.

In [10]:
jaro_replace_names(unique_cities, "philadelphia", 0.6)

philadelphia -> philadelphia, score: 1.00
philadelphia  -> philadelphia, score: 0.97
philiadelphia -> philadelphia, score: 0.97
philadephia -> philadelphia, score: 0.97
philadelphila  -> philadelphia, score: 0.95
philadelphia pa -> philadelphia, score: 0.93
philiidelphia -> philadelphia, score: 0.83
west philadelphia -> philadelphia, score: 0.82
phila -> philadelphia, score: 0.81
philadelphia (northeast philly) -> philadelphia, score: 0.80
philly -> philadelphia, score: 0.75
southwest philadelphia -> philadelphia, score: 0.71
phonixville -> philadelphia, score: 0.69
upland -> philadelphia, score: 0.67
pineville -> philadelphia, score: 0.67
landsdale -> philadelphia, score: 0.67
silverdale -> philadelphia, score: 0.64
holland -> philadelphia, score: 0.64
red hill -> philadelphia, score: 0.64
hatfield -> philadelphia, score: 0.64
ivyland -> philadelphia, score: 0.63
paoli -> philadelphia, score: 0.63
pipersville -> philadelphia, score: 0.63
springfield -> philadelphia, score: 0.63
lansda

Let's use 0.7 as a threshold for the Jaro score to replace different alias of Philadelphia for filtering the business dataset later. It seems that the algorithm worked correctly judging from the fact that the `assert` statement, which checks how many aliases of Philadelphia were reduced after normalising the city's name in `business_df`, did not return any errors. 

In [11]:
# Replacing the alias of Philadelphia in the dataset
old_unique_cities_len = len(unique_cities)
business_pa.loc[:, "city"] = business_pa['city'].apply(replace_name, new_name = 'philadelphia', threshold = 0.7)
new_unique_cities_len = len(business_pa['city'].unique())

# Checking we've normalised Philadelphia correctly
assert old_unique_cities_len - new_unique_cities_len == 11  # There were 11 aliases for Philadelphia in the original business_df dataset

### Checking Restaurant-related Businesses
We shall also pre-process the `categories` column to ensure that we do not miss out restaurant-related reviews due to typos. Using regex to look for words containing part of the spelling of restaurants (namely "Rest"), we see that more than one result were returned, of which 3 contain the term 'restaurant.' Again, we can use Jaro similarity to check whether restaurants are referred to with multiple ways due to typos.

Contrary to the case of the city's name, "restaurants" do not have many variants of how it is referred to in `business_df`. Indeed, only the categories of *restaurants* and *pop-up restaurants* seem to be relevant for helping the business owners understand the regional market of the catering industry. Therefore, we can just keep these two categories in `business_df`. 

In [12]:
from itertools import chain

# Getting all the distinct business categories 
business_pa.loc[:, "categories"] = business_pa["categories"].str.lower()
categories_split = chain(*list(business_pa["categories"].dropna().str.split(r",\s+", regex=True)))
unique_categories = list(set(categories_split))

# Checking which categories might contain part of the spelling of the word restaurant
resto_matches = [re.search(r"Rest.*", cat, flags=re.IGNORECASE) is not None for cat in unique_categories]
np.array(unique_categories)[resto_matches]

array(['restaurant supplies', 'pop-up restaurants', 'art restoration',
       'damage restoration', 'rest stops', 'restaurants'], dtype='<U35')

In [13]:
# Checking how many categories can be considered as variants of restaurant-related businesses
jaro_replace_names(unique_categories, "restaurants", threshold = 0.7)

restaurants -> restaurants, score: 1.00
restaurant supplies -> restaurants, score: 0.86
pop-up restaurants -> restaurants, score: 0.78
resorts -> restaurants, score: 0.75
austrian -> restaurants, score: 0.74
arts & crafts -> restaurants, score: 0.74
art space rentals -> restaurants, score: 0.73
party bus rentals -> restaurants, score: 0.73
vegetarian -> restaurants, score: 0.72
rest stops -> restaurants, score: 0.72
aerial tours -> restaurants, score: 0.71
real estate -> restaurants, score: 0.71
race tracks -> restaurants, score: 0.71
southern -> restaurants, score: 0.71
shutters -> restaurants, score: 0.71
attraction farms -> restaurants, score: 0.70
fire departments -> restaurants, score: 0.70


### Getting restaurant review data in Philly
After normalising the name of the city as well as finding out the business categories which correspond to restaurants, we can extract the data needed for subsequent analysis. We also dropped some columns which will not be useful to save memory.

In [14]:
# Getting businesses located in Philadelphia
business_philly = business_pa.query("city == 'philadelphia'").reset_index(drop = True)

# Getting only restaurant businesses in Philadelphia
restaurant_masks = business_philly["categories"].str.contains("restaurants") | business_philly["categories"].str.contains("pop-up restaurants")
restos_philly = business_philly.loc[restaurant_masks, :]

# Finally, joining user reviews with restaurant information
restos_philly = reviews_df.merge(restos_philly, how = "inner", on = "business_id")
assert np.sum(restos_philly['city'] == 'philadelphia') == restos_philly.shape[0]

# Dropping unnecessary columns
restos_philly.drop(columns = ["id", "business_id", "user_id", "city", "state"], inplace = True)

We now inspect the restaurant review data to see if more cleaning needs to be done. The data type of the `date` column is defined as `object` which will hinder our ability to analyse changes in customer preferences over time later, so let's convert it into `datetime` type object. By looking at the first few rows of the data, we also notice that the `attributes` column is in json format. It will be good to convert them into something easier to use for subsequent analysis if needed, namely by removing the symbols specific to json format with regex.

Missing values are also present in some columns of our data, and this is relatively the most serious for `hours`. Given the primary task is to make sense of customer preferences for restaurants in Philadelphia, opening hours are likely to be less informative than food style and service for deciding the optimal business strategy to open a new restaurant in the city. Accordingly, we will simply drop the `hours` column first, and then check how many rows will be omitted if we only include rows which contain non-null values in all the other columns.

In [15]:
# Inspecting the data types of the columns
print(restos_philly.info())
restos_philly.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103032 entries, 0 to 103031
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   rating       103032 non-null  int64  
 1   useful       103032 non-null  int64  
 2   funny        103032 non-null  int64  
 3   cool         103032 non-null  int64  
 4   text         103032 non-null  object 
 5   date         103032 non-null  object 
 6   name         103032 non-null  object 
 7   address      102899 non-null  object 
 8   postal_code  103027 non-null  object 
 9   latitude     103032 non-null  float64
 10  longitude    103032 non-null  float64
 11  attributes   102969 non-null  object 
 12  categories   103032 non-null  object 
 13  hours        99327 non-null   object 
dtypes: float64(2), int64(4), object(8)
memory usage: 11.0+ MB
None


,rating,useful,funny,cool,text,date,name,address,postal_code,latitude,longitude,attributes,categories,hours
0,4,0,0,0,One of my favorite places to grab a quick bit ...,2014-05-28 19:05:26,Chickie's & Pete's,1526 Packer Ave,19145,39.911417,-75.174511,"{'GoodForKids': 'True', 'Corkage': 'False', 'H...","seafood, nightlife, sports bars, bars, restaur...","{'Monday': '11:0-2:0', 'Tuesday': '11:0-2:0', ..."
1,5,0,0,0,I love this place! There's nothing like authen...,2013-05-28 16:52:23,Guacamole Mex-Grill,4612 Woodland Ave,19143,39.943971,-75.209914,"{'Caters': 'False', 'RestaurantsAttire': ""u'ca...","mexican, restaurants, specialty food, ethnic f...","{'Monday': '11:0-20:0', 'Tuesday': '11:0-20:0'..."
2,4,2,0,0,Situated on Race street away from the main str...,2014-12-21 15:03:49,Shiao Lan Kung,930 Race St,19107,39.955247,-75.155409,"{'RestaurantsGoodForGroups': 'True', 'Restaura...","noodles, seafood, restaurants, chinese","{'Tuesday': '15:0-2:0', 'Wednesday': '15:0-2:0..."
3,5,0,0,0,"Outstanding venue, great band, and the food is...",2015-10-17 01:07:26,Amari's Restaurant,5037 Baltimore Ave,19143,39.947985,-75.224744,"{'HasTV': 'True', 'RestaurantsAttire': ""u'casu...","soul food, american (new), breakfast & brunch,...","{'Wednesday': '7:30-15:0', 'Thursday': '7:30-1..."
4,5,4,0,2,We loved our experience at Elwood.\n\nWhen you...,2019-05-04 12:24:05,Elwood,1007 Frankford Ave,19125,39.966404,-75.134227,"{'RestaurantsReservations': 'True', 'OutdoorSe...","american (traditional), american (new), food, ...","{'Thursday': '17:0-22:0', 'Friday': '17:0-22:0..."


In [16]:
# Converting the date column into datetime data type for easier manipulation
restos_philly["date"] = pd.to_datetime(restos_philly['date']).dt.normalize()

# Cleaning attributes
restos_philly["attributes"] = restos_philly["attributes"].str.replace(r"\{|\}|'|\"|\bu", "", regex=True)
restos_philly["attributes"] = restos_philly["attributes"].str.split(", ")
restos_philly["attributes"][0][:5]  # I'll leave it like this for now but do further pre-processing if you need to

['GoodForKids: True',
 'Corkage: False',
 'HasTV: True',
 'RestaurantsAttire: casual',
 'RestaurantsTakeOut: True']

If we look at the shape of the `restos_philly` dataset after firsting dropping the `hours` columns and then the rows containing any null values, the decrease in the number of observations is only less than 200 which is minimal, so we will just proceed with this approach for dealing with missing values.

In [17]:
# Dropping the hours column 
restos_philly.drop(columns=['hours'], inplace = True)

# Checking how many obs we'll lose by dropping rows which contain missing values in any of the columns
restos_philly.dropna(inplace = True)  
restos_philly.info()  # The loss in observation is fewer than 200 rows which is minimal 

<class 'pandas.core.frame.DataFrame'>
Index: 102836 entries, 0 to 103031
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   rating       102836 non-null  int64         
 1   useful       102836 non-null  int64         
 2   funny        102836 non-null  int64         
 3   cool         102836 non-null  int64         
 4   text         102836 non-null  object        
 5   date         102836 non-null  datetime64[ns]
 6   name         102836 non-null  object        
 7   address      102836 non-null  object        
 8   postal_code  102836 non-null  object        
 9   latitude     102836 non-null  float64       
 10  longitude    102836 non-null  float64       
 11  attributes   102836 non-null  object        
 12  categories   102836 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(4), object(6)
memory usage: 11.0+ MB


Looking again at the `info()` of the dataset, every columns now has the correct data type, and the missing values are removed. Looking at the first 5 rows of the dataset, our pre-processing steps also seemed to work as intended. However, if we would like to use machine learning techniques (e.g. dimensionality reduction) on the textual data (namely, `text`, `categories` and potentially `attributes`), then we will need a way to convert them into numerical embeddings which can be understood by computers. We shall discuss this more when we analyse the data in later parts of this report.

In [18]:
# Format-wise the data look good
restos_philly.head()

,rating,useful,funny,cool,text,date,name,address,postal_code,latitude,longitude,attributes,categories
0,4,0,0,0,One of my favorite places to grab a quick bit ...,2014-05-28,Chickie's & Pete's,1526 Packer Ave,19145,39.911417,-75.174511,"[GoodForKids: True, Corkage: False, HasTV: Tru...","seafood, nightlife, sports bars, bars, restaur..."
1,5,0,0,0,I love this place! There's nothing like authen...,2013-05-28,Guacamole Mex-Grill,4612 Woodland Ave,19143,39.943971,-75.209914,"[Caters: False, RestaurantsAttire: casual, Noi...","mexican, restaurants, specialty food, ethnic f..."
2,4,2,0,0,Situated on Race street away from the main str...,2014-12-21,Shiao Lan Kung,930 Race St,19107,39.955247,-75.155409,"[RestaurantsGoodForGroups: True, RestaurantsTa...","noodles, seafood, restaurants, chinese"
3,5,0,0,0,"Outstanding venue, great band, and the food is...",2015-10-17,Amari's Restaurant,5037 Baltimore Ave,19143,39.947985,-75.224744,"[HasTV: True, RestaurantsAttire: casual, DogsA...","soul food, american (new), breakfast & brunch,..."
4,5,4,0,2,We loved our experience at Elwood.\n\nWhen you...,2019-05-04,Elwood,1007 Frankford Ave,19125,39.966404,-75.134227,"[RestaurantsReservations: True, OutdoorSeating...","american (traditional), american (new), food, ..."


In [19]:
restos_philly.describe()

,rating,useful,funny,cool,date,latitude,longitude
count,102836.000000,102836.000000,102836.000000,102836.000000,102836,102836.000000,102836.000000
mean,3.813032,1.096289,0.368460,0.544576,2015-12-06 03:08:51.409233920,39.961365,-75.160112
min,1.000000,0.000000,0.000000,0.000000,2005-05-26 00:00:00,39.865466,-75.363647
25%,3.000000,0.000000,0.000000,0.000000,2013-06-18 00:00:00,39.947539,-75.171477
50%,4.000000,0.000000,0.000000,0.000000,2016-03-23 00:00:00,39.950948,-75.161766
75%,5.000000,1.000000,0.000000,1.000000,2018-08-19 00:00:00,39.961212,-75.150048
max,5.000000,115.000000,82.000000,112.000000,2022-01-19 00:00:00,40.141488,-74.940729
std,1.303007,2.476457,1.378197,1.795871,NaN,0.035640,0.035608


In [20]:
# # Creating copy of the restaurant data
# batch_n = int(5e4)
# restos_philly.iloc[:batch_n, :].to_csv("restos_philly_batch1.csv", index=False)
# restos_philly.iloc[batch_n:, :].to_csv("restos_philly_batch2.csv", index=False)


# # Loading the pre-processed data in csv files
# restos_b1 = pd.read_csv("restos_philly_batch1.csv")
# restos_b2 = pd.read_csv("restos_philly_batch2.csv")
# restos_all = pd.concat([restos_b1, restos_b2])
# assert restos_all.shape == restos_philly.shape

## Exploring the restaurant reviews data

Now that we have extracted the restaurant review data in Philadelphia, let's start by some exploratory data analysis (EDA) to make sense of the data and (hopefully) find some inspirations of what advice can be given to the business owner. 

### How often are restaurants reviewed by customers?

To begin with, how often is each restaurant reviewed by customers in Philly? The histogram below indicates that very few restaurants have received more than 30 reviews. 

<!-- Some EDAs
1. Distribution of the count of reviews per resto
2. 
-->

<!-- Some ideas of the analysis: 
1. Use embeddings on reviews and categories and then run LSA to summarise them as "themes" (e.g., what is a review about in terms of food, service etc.? How can we group many categories into one based on their similarity?) 
2. Analyse how the themes are related to rating to understand what customers like/ don't like about
-->

In [21]:
# Number of reviews per restaurant
review_counts = restos_philly["name"].value_counts()

fig = px.histogram(review_counts, 
                   title = " Histogram of the number of customer reviews per restaurant in Philadelphia")

fig.update_layout(xaxis_title = "Number of customer reviews", 
                  yaxis_title = "Count of restaurants", 
                  showlegend = False)

fig.show()

In particular, which are the most frequently reviews restaurants in the city? And what are their average ratings and types of restaurant? We can observe from below that the cuisine styles of the top 20 most reviewed restaurants are quite diverse, as we've cafes, Chinese restaurants and steakhouses among other types. Furthermore, most of the restaurants receive average ratings of at least 4, which could imply that customers in Philadelphia tend to give their reviews to restaurants which they like, albeit there is an exception of Geno's Steaks which only has an average rating of around 2.5.

In [22]:
# Getting top 20 restaurants in terms of reviews received
top_20_counts = review_counts[:20]

# Getting average ratings of top 20 restos
top_20_restos = top_20_counts.index
top_20_avg_ratings = restos_philly.query("name in @top_20_restos").groupby("name")["rating"].mean()

top_20s = pd.DataFrame(data = {"reviews_received": top_20_counts, 
                               "average_rating": top_20_avg_ratings}, 
                       index = top_20_restos)
top_20s

,reviews_received,average_rating
name,,
Reading Terminal Market,888,4.603604
Pat's King of Steaks,624,3.205128
Green Eggs Café,560,3.882143
Sabrina's Café,530,4.173585
Dim Sum Garden,523,3.923518
El Vez,511,4.017613
Geno's Steaks,505,2.493069
Barbuzzo,440,4.279545
Zahav,439,4.526196


Below we also look at the proportion of reviews about the top 20 most reviewed restaurants out of all the restaurants in the dataset. They altogether received about 9% of the reviews from customers in Philadelphia, which is not a small proportion.

In [23]:
unique_restos = restos_philly["name"].unique()
n_restos = len(unique_restos)
print(f"There are {n_restos} restaurants in Philadelphia under the current dataset, and the top 20 most reviewed restaurants received {sum(top_20s["reviews_received"]) / restos_philly.shape[0] * 100:.2f}% of reviews.")

There are 4683 restaurants in Philadelphia under the current dataset, and the top 20 most reviewed restaurants received 8.84% of reviews.


### A brief look on the evolution of ratings over time

Apart from the distribution of the number of reviews among restaurants, we shall also explore how the customer ratings evolved throughout the period covered by the data. The dates of the reviews are originally recorded on the second level which will be too fine grained for understanding the rating trends. Therefore, we will start by aggregating the dates on a monthly level. the restaurant reviews covered in this dataset were created from May 2005 to January 2022.

In [24]:
# Aggregating dates to monthly level
restos_philly["date"] = restos_philly["date"].dt.to_period("M").dt.to_timestamp()
restos_philly["date"].describe()

count                           102836
mean     2015-11-21 10:41:11.508032256
min                2005-05-01 00:00:00
25%                2013-06-01 00:00:00
50%                2016-03-01 00:00:00
75%                2018-08-01 00:00:00
max                2022-01-01 00:00:00
Name: date, dtype: object

It will also be useful to look at how the average ratings evolved throughout this period, and more importantly how many reviews (or observations) there are for each month in the dataset, since this will affect how certain we can say about the food preference of customers in a given month. 

In the line plot on the left below, the monthly average rating throughout the period fluctuated around 3.7, albeit there was some erratic movement up to the end of 2007. What could be the reasons? The line plot on the right showing the number of reviews per month might give us an answer, namely, there were very few (sometimes even only a single digit) reviews for each month until December 2007, and thus the variances of the average rating for these months will be higher than subsequent months starting from 2008 when there were substantively more observations. We shall bear this characteristic in mind as we analyse the evolution of food trends in later sections, and we will discuss how to handle this accordingly.

Another interesting observations from the right line plot is that the number of reviews had a sharp drop since February 2020. Given how countries (including the US) responded to the COVID-19 outbreak by lockdowns (one of the measures being the prohibition of dine-in in restaurants), it is then not surprising that customers had fewer (or even no) chances to visit restaurants and thus not able to give as many reviews as before the COVID outbreak. 

In [25]:
# Seeing how the average ratings evolved as a function of time
from plotly.subplots import make_subplots

monthly_avg_rating = restos_philly.groupby("date")["rating"].apply(np.mean)
monthly_rating_no = restos_philly.groupby("date")["rating"].count()

monthly_trend = pd.DataFrame({"review_no": monthly_rating_no, 
                              "avg_rating": monthly_avg_rating})

fig = make_subplots(rows = 1, cols = 2)

fig.add_trace(
    go.Scatter(x = monthly_trend.index,
               y = monthly_trend["avg_rating"], 
               mode = "lines", showlegend = False), 
    row = 1, col = 1
)

fig.add_trace(
    go.Scatter(x = monthly_trend.index,
               y = monthly_trend["review_no"], 
               mode = "lines", showlegend = False), 
    row = 1, col = 2
)

fig.update_layout(title_text = "Trends of customer reviews on restaurants from May 2005 to January 2022 in Philadelphia",
                  xaxis1_title = "Time", yaxis1_title = "Average rating", 
                  xaxis2_title = "Time", yaxis2_title = "Number of reviews")

fig.show()

## The evolution of food trends over time

Besides looking at what customers in general like about restaurants, it will be also beneficial to understand whether food preferences by customers might change over time. For instance, there might exist "seasonality" for certain types of cuisine, as fondue 🫕 is usually more preferred by customers in winters than in summers. Having knowledge of food trends in Philadelphia can be useful for the business owner to decide which type of restaurant to open and whether he will have to adjust the restaurant's capacity for service depending on the season. 

But how should we define "food trends" with the data we are using? Recall that the `categories` column, now that only restaurant-type businesses are included, contain information about the types of catering services or cuisine a restaurant offers. Accordingly, we will see how the average ratings for categories of restaurants changed over the period of time covered in the dataset. 

Continuing from where we left off about the number of reviews for each month in the dataset, we will only analyse the food trends with data from 2008 because the earlier period in the dataset contains too few observations in each month for us to make any reliable conclusions about the food trends. But first, let's put our attention to how we can summarise the categories of the restaurants.

### Clustering restaurants into different types 
Naively, one might just want to directly work on the categories of the restaurants to examine food trends. On a closer look, however, there are over 300 unique categories describing businesses with restaurants being one of their services in Philadelphia. Using the categories directly will then make presenting the food trends in a concise manner to the business owner difficult.

In [334]:
# Checking how many categories of restaurants there are in the dataset
restos_philly["categories"] = restos_philly["categories"].str.split(r",\s+", regex = True)
resto_cat_list = restos_philly["categories"].to_list()
unique_resto_types = set(chain(*resto_cat_list))
print(f"There are a total of {len(unique_resto_types)} categories related to restaurants in the dataset.")

There are a total of 367 categories related to restaurants in the dataset.


We would therefore need a way to summarise all the restaurants in Philadelphia as a few major types and then look at how the ratings have evolved for them over the period covered in the dataset. Using Cluster analysis, which is one of the unsupervised learning algorithms, to find similar groups (the number being pre-defined) in the data will be very helpful for this task. The first step is then to convert the restaurants, whose categories are in texts in the dataset, into some representations that can be understood by unsupervised learning algorithms which use numerical inputs. Word embeddings can help with this. 

Specifically, below are the steps of using cluster analysis to summarise the restaurants into a few major types:

1. Train word embeddings for the categories of restaurant-related businesses
2. Use the above word embeddings to create an embedding vector of each restaurant
3. Perform clustering of the restaurants and see how many groups fit the restaurants best

For word embeddings, we will opt for the Word2Vec algorithm and train it on the restaurant categories in the dataset. A brief explanation of the above decision: we are training a Word2Vec model here because our task is quite specific, namely, understanding the food preferences of customers in Philadelphia on several major types of restaurants. If we use pre-trained embedding models, then we will likely risk not being able to understand how some categories with other usages in English just mean for restaurant categories. For instance, "Chinese" is one category of restaurants included in the dataset, but it can also be used in other contexts such as politics and media, and it is very likely that pre-trained models (usually on a large corpus of documents from the Internet) will not have embeddings for the word "Chinese" solely on its meaning as a cuisine style.

As for using the Word2Vec algorithm, one of the reasons is that it can be easily trained with personal computers via `gensim`, unlike GloVe which requires using the command line to train or Transformers architectures which are prohibitively expensive for individuals to train on their own computers. Moreover, Word2Vec can capture more nuances of the restaurant categories beyond their similarities than rudimentary embedding methods like the bag of words or TF-IDF. 

Since Word2Vec is already discussed in detail during module 7 of this course, here we will just quickly mention its main characteristics.  

In [335]:
# For training word embeddings and clustering
from gensim.models import Word2Vec
from sklearn.cluster import KMeans

# For faster computation
import multiprocessing
cores = multiprocessing.cpu_count()

Below we extract the unique entries of restaurants in the dataset along with their categories. We notice that the extracted data below have more rows than the total number of restaurants reviewed in Philadelphia in our dataset. What is the reason behind? 

In [336]:
# Getting all the distinct restaurants and their respective categories
restos_philly["categories"] = restos_philly["categories"].apply(tuple)  # To reduce duplicates due to the same categories appearing in different orders
resto_cats = restos_philly[["name", "categories"]].drop_duplicates()
resto_cats.shape

(5285, 2)

It seems that there are restaurants which have more than one entry of their categories in the data, and if we look at the case of Popeyes Louisiana Kitchen, then we observe that some categories are present at some point in the data but not in others, causing multiple entries of categories for certain restaurants. We can combine these repeated entries into one by using `set`.

In [337]:
# Restaurants having more than one entry for categories
restaurant_counts = resto_cats["name"].value_counts()
print(restaurant_counts)

popeyes = resto_cats.query("name == 'Popeyes Louisiana Kitchen'")
popeyes

name
Wawa                         35
McDonald's                   30
Dunkin'                      20
Popeyes Louisiana Kitchen    16
Taco Bell                    14
                             ..
Tiki Philly                   1
Café y Chocolate              1
Lucha Cartel                  1
Cheu Fishtown                 1
DOHO Taqueria                 1
Name: count, Length: 4683, dtype: int64


,name,categories
1461,Popeyes Louisiana Kitchen,"(american (traditional), fast food, restaurant..."
2911,Popeyes Louisiana Kitchen,"(restaurants, fast food, american (traditional..."
5192,Popeyes Louisiana Kitchen,"(american (traditional), fast food, food, chic..."
12532,Popeyes Louisiana Kitchen,"(fast food, southern, restaurants, american (t..."
13475,Popeyes Louisiana Kitchen,"(burgers, american (traditional), chicken wing..."
16801,Popeyes Louisiana Kitchen,"(restaurants, southern, fast food, chicken win..."
24067,Popeyes Louisiana Kitchen,"(american (traditional), restaurants, chicken ..."
26345,Popeyes Louisiana Kitchen,"(chicken wings, restaurants, american (traditi..."
26439,Popeyes Louisiana Kitchen,"(american (traditional), restaurants, chicken ..."
30957,Popeyes Louisiana Kitchen,"(chicken wings, american (traditional), fast f..."


In [338]:
# Concatenating all distinct categories for each restaurant
def combine_cats(restaurant):
    categories = resto_cats.query("name == @restaurant")["categories"].to_list()
    return set(chain(*categories))

resto_cats_dict = {}

for resto in restaurant_counts.index:
    resto_cats_dict[resto] = combine_cats(resto)
    
resto_cats = pd.DataFrame({"name": resto_cats_dict.keys(),
                           "categories": resto_cats_dict.values()})
assert resto_cats.shape[0] == n_restos
resto_cats.head()

,name,categories
0,Wawa,"{fast food, automotive, food, convenience stor..."
1,McDonald's,"{fast food, food, burgers, coffee & tea, ameri..."
2,Dunkin',"{donuts, food, bagels, coffee & tea, breakfast..."
3,Popeyes Louisiana Kitchen,"{chicken wings, fast food, cajun/creole, food,..."
4,Taco Bell,"{fast food, mexican, breakfast & brunch, resta..."


In [339]:
# A function for converting restaurants into embedding vectors based on its categories
def resto2vec(tokens, embedding_wv, normalize=True):
    """Returns the embedding of a restaurant-related business as the mean of the tokens/words embeddings of its categories."""
    sent_mean = np.array([embedding_wv.get_vector(tok, norm=normalize) for tok in tokens]).mean(axis=0)
    return sent_mean

In [342]:
w2v_resto = Word2Vec(resto_cat_list, vector_size = 30, window = 2, min_count = 5, sg = 1, negative = 2, ns_exponent = 0.75,
                     alpha = 0.01, min_alpha = 0.0001, workers = cores - 1, epochs = 30, seed = 123)

In [343]:
w2v_resto.wv.most_similar("chinese")

[('taiwanese', 0.6775922775268555),
 ('noodles', 0.5927032828330994),
 ('do-it-yourself food', 0.5586516857147217),
 ('dim sum', 0.5480829477310181),
 ('health markets', 0.5426425337791443),
 ('pan asian', 0.5326589941978455),
 ('asian fusion', 0.5249775648117065),
 ('thai', 0.521957516670227),
 ('shanghainese', 0.5096319913864136),
 ('szechuan', 0.5070323944091797)]